# Exploring the methods of LyaCoLoRe

The aim of this notebook is to show the stages LyaCoLoRe goes through when converting CoLoRe's Gaussian skewers to realistic skewers of transmitted flux fraction. There are 5 main stages which we will go through, showing an example skewer at each stage to illustrate.

In this notebook, we do not compute additional astrophysical effects such as metals and DLAs. Those will be addressed separately.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.interpolate import interp1d

from lyacolore.simulation_data import SimulationData
from lyacolore.independent import power_kms
from lyacolore import utils

In [ ]:
# Define some plotting parameters: the figure size, font size and some nice colours
figsize = (12,8)
fontsize = 18; plt.rc('font', size=fontsize)
colours = C0,C1,C2,C3 = ['#F5793A','#A95AA1','#85C0F9','#0F2080']

In [ ]:
# Define a random seed to control the random elements of the mock-making process. 
# Then make a RandomState generator object.
seed = 0
generator = np.random.RandomState(seed)

In [ ]:
# Define which range of wavelengths we would like to show in our plots.
# A range of ~100Å is generally about right.
lambda_min = 3800.
lambda_max = 3900.

In [ ]:
# Define the location of the input CoLoRe file, and the transformation parameter file.
f_colore = os.environ['LYACOLORE_PATH']+'/example_data/gaussian/colore_output/out_srcs_s1_0.fits'
f_tuning = os.environ['LYACOLORE_PATH']+'/input_files/tuning_files/tuning_data_v9.0.fits'

## Stage 0: Load the input data
We need to load the input data: an output file from `CoLoRe` and a file containing the parameters to use when transforming our skewers. The latter of these is generated by `LyaCoLoRe`'s tuning process, though that is not described here. 

First, we create a `SimulationData` object, from which we can then retrieve all of the relevant information.

In [ ]:
skw_obj = SimulationData.get_skewers_object(f_colore,0,'colore','gaussian')

With this object, we can do many things. For example, we can explore the redshift distribution of our QSOs:

In [ ]:
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
_ = axs[0,0].hist(skw_obj.Z_QSO,bins=np.linspace(1.,4.,31),histtype='step',color=C0)
_ = axs[0,0].set_xlabel('z')
_ = axs[0,0].set_ylabel('#')

Or we can look at an example skewer:

In [ ]:
# Choose which skewer we want to look at (integer between 0 and 999)
i_skw = 0

In [ ]:
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
wavelength = (1+skw_obj.Z)*utils.lya_rest
_ = axs[0,0].plot(wavelength,skw_obj.GAUSSIAN_DELTA_rows[i_skw,:],label=r'$\delta_C$',c=C0,ls='--')
_ = axs[0,0].set_xlim(lambda_min,lambda_max)
_ = axs[0,0].set_ylim(-4,4)
_ = axs[0,0].set_xlabel(r'$\lambda\ [\mathrm{Å}]$')
_ = axs[0,0].legend()

We now add the transformation parameters to the `SimulationData` object, loading them from file. These parameters are described in equations (2.2) to (2.5) of https://arxiv.org/abs/1912.02763.

In [ ]:
# Add the transformation parameters from file.
skw_obj.add_transformation_from_file(f_tuning)

We now reduce our `SimulationData` object to contain only one skewer, and to remove cells that are at low redshift (and so do not contribute to Lya forest). This ensures that the code runs at top speed!

In [ ]:
# Include only cells above z=1.8
z_trim = 1.8 
lambda_trim = utils.lya_rest*(1+z_trim)
skw_obj.trim_skewers(lambda_min=lambda_trim,remove_irrelevant_QSOs=True)

# Include only the skewer corresponding to i_skw.
skw_obj.reduce_nskw(MOCKIDs=skw_obj.MOCKID[i_skw])

## Stage 1: Add small-scale fluctuations
In order to obtain realistic errors on BAO measurements from our mocks, we need out skewers to have approximately correct 1D power (on certain scales). However, our output skewers from CoLoRe have very large cells - ~2.4Mpc/h in this case - which makes obtaining the correct 1D power impossible. 

As such, we need to convert our skewers to use a smaller cell size, and add fluctuations on these smaller scales in order to obtain the correct 1D power.

First, set the new cell size that we would like in Mpc/h.

In [ ]:
new_cell_size = 0.25 #Mpc/h

Now we may look at the shape of the extra 1D power that our transformation parameters define. The shape of this 1D power is defined by equation (2.2) in https://arxiv.org/pdf/1912.02763.pdf.

In [ ]:
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)

# Plot the 1D power spectrum we add to the skewers.
k_kms = np.logspace(-3,-1.5,201)
print('INFO: computing 1D power with n={:1.3f}, k1={:1.3f}'.format(skw_obj.transformation.n,skw_obj.transformation.k1))
pk_kms = power_kms(0.,k_kms,100*new_cell_size,n=skw_obj.transformation.n,k1=skw_obj.transformation.k1,R_kms=skw_obj.transformation.R_kms)
_ = axs[0,0].loglog(k_kms,pk_kms)
_ = axs[0,0].set_xlabel(r'$k\ [\mathrm{s\ km}^{-1}]$')
_ = axs[0,0].set_ylabel(r'$P_{1D}(k)\ [\mathrm{km\ s}^{-1}]$')
_ = axs[0,0].set_title('Extra 1D Power')

Now we add small-scale fluctuations to our skewers. We first interpolate the existing `CoLoRe` skewers onto the smaller cell size using NGP interpolation. We then generate random fluctuations according to the power spectrum above, with each skewer treated independently. These fluctuations are then scaled by a $z$-dependent function $\sigma_\epsilon(z)$ and added to the interpolated `CoLoRe` skewers (see equation (2.3) in https://arxiv.org/pdf/1912.02763.pdf)

In [ ]:
# Store a skewer before adding fluctuations for comparison later.
colore_wavelength = (1+skw_obj.Z)*utils.lya_rest
colore_skewer = skw_obj.GAUSSIAN_DELTA_rows[0,:]

# Add small-scale fluctuations to the Gaussian skewers.
skw_obj.add_small_scale_fluctuations(new_cell_size,generator,use_transformation=True)

The difference between the `CoLoRe` skewers (labelled $\delta_C$) and the new skewers (labelled $\delta_G$) is clear to see.

In [ ]:
# Show the difference before and after adding the small-scale fluctuations.
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
wavelength = (1+skw_obj.Z)*utils.lya_rest
_ = axs[0,0].plot(wavelength,skw_obj.GAUSSIAN_DELTA_rows[0,:],label=r'$\delta_G$',c=C0,ls='-')
_ = axs[0,0].plot(colore_wavelength,colore_skewer,label=r'$\delta_C$',c=C0,ls='--')
_ = axs[0,0].set_xlim(lambda_min,lambda_max)
_ = axs[0,0].set_ylim(-20,20)
_ = axs[0,0].set_xlabel(r'$\lambda\ [\mathrm{Å}]$')
_ = axs[0,0].legend()

## Stage 2: Convert to physical density
We now want to convert the Gaussian skewers (with small-scale fluctuations) to skewers of physical density. We do so using a lognormal transformation (see equation (2.4) of https://arxiv.org/pdf/1912.02763.pdf). We compute skewers of this density $\delta$, and plot the same example skewer as previously.

In [ ]:
skw_obj.compute_physical_skewers()

In [ ]:
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
_ = axs[0,0].plot(wavelength,skw_obj.DENSITY_DELTA_rows[0,:],label=r'$\delta$',c=C1,ls='-')
_ = axs[0,0].set_xlim(lambda_min,lambda_max)
_ = axs[0,0].set_ylim(-2,20)
_ = axs[0,0].set_xlabel(r'$\lambda\ [\mathrm{Å}]$')
_ = axs[0,0].legend()

## Stage 3: Convert to optical depth
We now convert to optical depth using the fluctuating Gunn-Peterson approximation (see equation (2.5) of https://arxiv.org/pdf/1912.02763.pdf). This involves two transformation parameters, which we name $\tau_0(z)$ and $\alpha(z)$ and plot below (along with $\sigma_\epsilon(z), which we saw earlier).

In [ ]:
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)

# Plot the transformation parameters as a function of z.
z = np.linspace(2.,4.,201)
_ = axs[0,0].plot(z,skw_obj.transformation.f_seps_z(z),label=r'$\sigma_{\epsilon}$')
_ = axs[0,0].plot(z,skw_obj.transformation.f_texp_z(z),label=r'$\alpha$')
_ = axs[0,0].plot(z,skw_obj.transformation.f_tau0_z(z),label=r'$\tau_{0}$')
_ = axs[0,0].legend()
_ = axs[0,0].set_xlabel(r'$z$')
_ = axs[0,0].set_title(r'$z$-dependent transformation parameters')

We compute skewers of $\tau$, and plot the same example skewer as previously.

In [ ]:
skw_obj.compute_all_tau_skewers()

In [ ]:
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
wavelength = (1+skw_obj.Z)*utils.lya_rest
_ = axs[0,0].plot(wavelength,skw_obj.lya_absorber.tau[0,:],label=r'$\tau_{\mathrm{noRSD}}$',c=C2,ls=':')
_ = axs[0,0].set_xlim(lambda_min,lambda_max)
_ = axs[0,0].set_ylim(-1,30)
_ = axs[0,0].set_xlabel(r'$\lambda\ [\mathrm{Å}]$')
_ = axs[0,0].legend()

## Stage 4: Add RSDs
We now use the velocity skewers provided by `CoLoRe` to add redshift-space distortions (RSDs) to our skewers of $\tau$. We first apply an ad hoc rescaling of the velocity by a factor of 1.3 (tuned to acheive the correct value of $\beta_\mathrm{Lya}$. When we visualise the `CoLoRe` velocities (given in terms of redshift deviations $\Delta z$), it appears in "steps" as it has been NGP-interpolated when we added small scale fluctuations.

In [ ]:
skw_obj.scale_velocities(use_transformation=True)

In [ ]:
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
wavelength = (1+skw_obj.Z)*utils.lya_rest
_ = axs[0,0].plot(wavelength,skw_obj.VEL_rows[0,:],label=r'$\Delta z$',c='grey',ls='-')
_ = axs[0,0].set_xlim(lambda_min,lambda_max)
_ = axs[0,0].set_ylim(-0.01,0.01)
_ = axs[0,0].set_xlabel(r'$\lambda\ [\mathrm{Å}]$')
_ = axs[0,0].legend()

We then compute the RSD weights matrices (see appendix B of https://arxiv.org/pdf/1912.02763.pdf), which move cells up/down the skewer according to their velocity. When we look at the $\tau$ skewers, we can see how the cells have moved: peaks where the velocity is negative have moved to the left, and vice versa.

In [ ]:
skw_obj.add_all_RSDs()

In [ ]:
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
wavelength = (1+skw_obj.Z)*utils.lya_rest
_ = axs[0,0].plot(wavelength,skw_obj.lya_absorber.tau[0,:],label=r'$\tau$',c=C2,ls='-')
_ = axs[0,0].plot(wavelength,skw_obj.lya_absorber.tau_noRSD[0,:],label=r'$\tau_{\mathrm{noRSD}}$',c=C2,ls=':')
_ = axs[0,0].set_xlim(lambda_min,lambda_max)
_ = axs[0,0].set_ylim(-1,30)
_ = axs[0,0].set_xlabel(r'$\lambda\ [\mathrm{Å}]$')
_ = axs[0,0].legend()

## Stage 5: Convert to transmitted flux fraction
Finally, we convert to transmitted flux fraction. This provides our final skewers!

In [ ]:
f_skw = skw_obj.lya_absorber.transmission()

In [ ]:
fig, axs = plt.subplots(1,1,figsize=figsize,squeeze=False)
wavelength = (1+skw_obj.Z)*utils.lya_rest
_ = axs[0,0].plot(wavelength,f_skw[0,:],label=r'F',c=C3,ls='-')
_ = axs[0,0].set_xlim(lambda_min,lambda_max)
_ = axs[0,0].set_ylim(-0.05,1.05)
_ = axs[0,0].set_xlabel(r'$\lambda\ [\mathrm{Å}]$')
_ = axs[0,0].legend()